In [42]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
m = gp.Model("CABS Route Optimization")
m.Params.LogToConsole = 1 
m.Params.TimeLimit = 100
n = 26
stops = [0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
x = m.addVars(n,n,4, lb=0, name="x", vtype = GRB.BINARY)
t = m.addVars(n, 4, name = "t", vtype = GRB.INTEGER)

c = np.array([[0,7,8,8,8,12,14,12,5,4,1,10,12,10,5,14,14,14,13,10,10,10,7,7,7,9],
            [7,0,1,1,1,5,3,2,1,2,7,8,10,7,5,10,10,10,11,7,7,7,4,4,4,6],
            [8,1,0,0,0,3,6,2,3,4,8,10,12,8,5,9,9,9,10,8,8,8,5,5,5,7],
            [8,1,0,0,0,3,6,2,3,4,8,10,12,8,5,9,9,9,10,8,8,8,5,5,5,7],
            [8,1,0,0,0,3,6,2,3,4,8,10,12,8,5,9,9,9,10,8,8,8,5,5,5,7],
            [12,5,3,3,3,0,5,5,5,7,11,14,16,12,9,8,8,8,10,13,13,13,9,9,9,11],
            [14,3,6,6,6,5,0,2,5,7,11,13,15,14,11,3,3,3,4,7,7,7,10,10,10,15],
            [12,2,2,2,2,5,2,0,4,6,9,11,13,9,6,7,7,7,8,11,11,11,7,7,7,9],
            [5,1,3,3,3,5,5,4,0,2,6,7,9,6,3,10,10,10,9,6,6,6,3,3,3,6],
            [4,2,4,4,4,7,7,6,2,0,4,6,8,4,1,12,12,12,11,8,8,8,5,5,5,4],
            [1,7,8,8,8,11,11,9,6,4,0,9,11,9,4,13,13,13,12,9,9,9,6,6,6,8],
            [10,8,10,10,10,14,13,11,7,6,9,0,3,5,3,17,17,17,16,14,14,14,11,11,11,1],
            [12,10,12,12,12,16,15,13,9,8,11,3,0,2,4,19,19,19,18,12,12,12,10,10,10,5],
            [10,7,8,8,8,12,14,9,6,4,9,5,2,0,3,18,18,18,17,11,11,11,9,9,9,6],
            [5,5,5,5,5,9,11,6,3,1,4,3,4,3,0,15,15,15,14,10,10,10,8,8,8,1],
            [14,10,9,9,9,8,3,7,10,12,13,17,19,18,15,0,0,0,2,5,5,5,7,7,7,16],
            [14,10,9,9,9,8,3,7,10,12,13,17,19,18,15,0,0,0,2,5,5,5,7,7,7,16],
            [14,10,9,9,9,8,3,7,10,12,13,17,19,18,15,0,0,0,2,5,5,5,7,7,7,16],
            [13,11,10,10,10,10,4,8,9,11,12,16,18,17,14,2,2,2,0,3,3,3,5,5,5,14],
            [10,7,8,8,8,13,7,11,6,8,9,14,12,11,10,5,5,5,3,0,0,0,3,3,3,12],
            [10,7,8,8,8,13,7,11,6,8,9,14,12,11,10,5,5,5,3,0,0,0,3,3,3,12],
            [10,7,8,8,8,13,7,11,6,8,9,14,12,11,10,5,5,5,3,0,0,0,3,3,3,12],
            [7,4,5,5,5,9,10,7,3,5,6,11,10,9,8,7,7,7,5,3,3,3,0,0,0,9],
            [7,4,5,5,5,9,10,7,3,5,6,11,10,9,8,7,7,7,5,3,3,3,0,0,0,9],
            [7,4,5,5,5,9,10,7,3,5,6,11,10,9,8,7,7,7,5,3,3,3,0,0,0,9],
            [9,6,7,7,7,11,15,9,6,4,8,1,5,6,1,16,16,16,14,12,12,12,9,9,9,0]])



m.setObjective(gp.quicksum(x[i,j,k]*c[i,j] for i in range(n) for j in range(n) for k in range(0,4)) , GRB.MINIMIZE)

# Vehicle Leaves Node it Enters
for k in range(0,4):
    for i in range(0,n):
        m.addConstr(gp.quicksum(x[i,j,k] for j in range(0,n))==gp.quicksum(x[j,i,k] for j in range(0,n)))

# Ensure that evey Node is entered once
    for i in stops:
        m.addConstr(gp.quicksum(x[i,j,k] for j in stops for k in range(0,4))==1)

# Every Vehicle Leaves the Depot        
for k in range(0,4):
    m.addConstr(gp.quicksum(x[6,j,k] for j in stops)==1)

# Busses cannot leave to go to the stop it is at
for i in range(0,26):
    m.addConstr(gp.quicksum(x[i,i,k] for i in range(0,26) for k in range(0,4))==0)
    
#Dummy variables for repeated stops are only visited once by each bus
for k in range(0,4):
    m.addConstr(gp.quicksum(x[i,j,k] for i in range(0,26) for j in [2,3,4])==1)
    m.addConstr(gp.quicksum(x[i,j,k] for i in range(0,26) for j in [15,16,17])==1)
    m.addConstr(gp.quicksum(x[i,j,k] for i in range(0,26) for j in [19,20,21])==1)
    m.addConstr(gp.quicksum(x[i,j,k] for i in range(0,26) for j in [22,23,24])==1)

Set parameter TimeLimit to value 100


In [43]:
# Eliminate Subtours
for k in range(0,4):
    for i in stops:
        for j in stops:
            if (i!=j):
                m.addConstr(t[j,k]-t[i,k] >= 1-(14*(1-x[i,j,k])))
                
for k in range(0,4):                
    for i in stops:
        m.addConstr(t[i,k] >= 1)
        m.addConstr(t[i,k] <= 14)

In [44]:
m.optimize()

print("obj_func = ", m.objVal)
for v in m.getVars():
    if v.x == 1:
        print('%s = %g' % (v.varName, v.x))

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2850 rows, 2808 columns and 26652 nonzeros
Model fingerprint: 0x42058182
Variable types: 0 continuous, 2808 integer (2704 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 397 rows and 204 columns
Presolve time: 0.28s
Presolved: 2453 rows, 2604 columns, 15328 nonzeros
Variable types: 0 continuous, 2604 integer (2504 binary)

Root relaxation: objective 9.700000e+01, 369 iterations, 0.12 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   97.00000    0   48          -   97.00000      -     -    0s
     0   

In [2]:
for i in range(26):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
